In [1]:
import pandas as pd 
import numpy as np
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
# Define a simple 1D diffusion model for sine wave generation
class SimpleDiffusion:
    def __init__(self, timesteps=1000):
        self.timesteps = timesteps
        
        # Create noise schedule (beta values)
        self.betas = torch.linspace(0.0001, 0.02, timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_cumprod = torch.cumprod(self.alphas, dim=0)
        
    def add_noise(self, x, t):
        """Add noise to data at timestep t"""
        noise = torch.randn_like(x)
        sqrt_alpha_cumprod = torch.sqrt(self.alpha_cumprod[t])
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alpha_cumprod[t])
        
        return sqrt_alpha_cumprod * x + sqrt_one_minus_alpha_cumprod * noise, noise

# Generate sample sine wave data
def generate_sine_data(num_samples=1000, seq_length=100):
    x = np.linspace(0, 4*np.pi, seq_length)
    data = []
    
    for _ in range(num_samples):
        # Add some variation to frequency and phase
        freq = np.random.uniform(0.5, 2.0)
        phase = np.random.uniform(0, 2*np.pi)
        amplitude = np.random.uniform(0.5, 1.5)
        
        sine_wave = amplitude * np.sin(freq * x + phase)
        data.append(sine_wave)
    
    return torch.tensor(np.array(data), dtype=torch.float32)

# Create dataset
sine_data = generate_sine_data()
diffusion = SimpleDiffusion()

print(f"Generated sine data shape: {sine_data.shape}")
print("Diffusion model initialized with {} timesteps".format(diffusion.timesteps))

Generated sine data shape: torch.Size([1000, 100])
Diffusion model initialized with 1000 timesteps


In [6]:
# Define a simple 1D diffusion model for sine wave generation
class SimpleDiffusion:
    def __init__(self, timesteps=1000):
        self.timesteps = timesteps
        
        # Create noise schedule (beta values)
        self.betas = torch.linspace(0.0001, 0.02, timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_cumprod = torch.cumprod(self.alphas, dim=0)
        
    def add_noise(self, x, t):
        """Add noise to data at timestep t"""
        noise = torch.randn_like(x)
        sqrt_alpha_cumprod = torch.sqrt(self.alpha_cumprod[t]).unsqueeze(1)  # Add dimension for broadcasting
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alpha_cumprod[t]).unsqueeze(1)  # Add dimension for broadcasting
        
        return sqrt_alpha_cumprod * x + sqrt_one_minus_alpha_cumprod * noise, noise

# Update the diffusion object
diffusion = SimpleDiffusion()


In [7]:
diffusion_data = []
# Add noise to the sine data at each timestep
for t in range(diffusion.timesteps):
    noisy_data, noise = diffusion.add_noise(sine_data, t)
    diffusion_data.append(noisy_data)
# Convert to tensor
diffusion_data = torch.stack(diffusion_data)
print(f"Diffusion data shape: {diffusion_data.shape}")
#plot the first few noisy samples
import matplotlib.pyplot as plt
for i in range(5):
    plt.plot(diffusion_data[:, i].numpy(), label=f'Sample {i+1}')
plt.title('Noisy Sine Wave Samples')
plt.xlabel('Time Steps')
plt.ylabel('Amplitude')
plt.legend()
plt.show()


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)